# 1 Leiðarval hjá Strætó
Í þessu verkefni á að búa til leitarvél fyrir Strætó, notast verður við raungögn frá Strætó
sem er að finna á
https://straeto.is/um-straeto/opin-rafraen-gogn.
Til einföldunar munuð þið nota skrána gtfs.zip sem er að finna á Canvas síðu námskeiðsins til að tryggja að
allir noti sömu útgáfu.


## 2.1 Inntak
Inntakið er tímatafla strætó, listi yfir leiðir, vagna og stoppistöðvar. Við skilgreinum netið
G á eftirfarandi hátt. Hver einasta stoppistöð verður hnútur.
Ef vagn ekur á milli stoppistöðva x og y á tímum t1 og t2 með engum öðrum stoppum á
milli þá búum við til nýjan hnút v með leggina (x, v), (v, y) og merkjum þá með tímunum t_1 og t_2. Hnúturinn v geymir upplýsingar um hvaða vagni sá hnútur tilheyrir, athugið þessi
hnútur verður ekki endurnýttur.
Fyrir leiðina sem er sýnd á myndinni þá samsvarar hún fimm hnútum. Hnútar 1, 3, 5
eru stoppistöðvarnar Hlemmur B, Sólfarið og Harpa. Hnútur 2 tengir saman Hlemm B og
Sólfarið og leggirnir hafa tímann 10:22 og 10:25. Hnútur 4 tengir saman Sólfarið og Hörpu,
fyrri leggurinn hefur tímann 10:25 (ekki sýnilegt á mynd) og sá seinni 10:26. Þetta er dæmi
um löglegan veg þar sem allir leggir samsvara vögnum sem við getum tekið og tímarnir
stemma, þ.e. eru í vaxandi röð

## 2.2 Reiknirit
Einfaldasta tilfellið sem við viljum leysa þá fá um við par af stoppistöðvum og tíma, (x, y, t)
og úttakið er leið í strætókerfinu frá x til y sem fer frá x á tíma t′ ≥ t og lágmarkar
komutímann á áfangastað. Ef við skiptum um strætisvagn þá samsvarar það því að "bíða"á
hnútnum sem er á stoppistöð og taka legg sem tilheyrir öðrum vagni heldur en þeim sem
við komum með.
Til að leysa þetta verkefni er hægt að nota reiknirit Dijkstra með einni breytingu, í stað
þess að slaka (RELAX aðferðin) á leggjum með því að leggja saman vigtir þá notum við
komutímann á leggnum ef leggurinn er löglegur, þ.e. ef (u, v) er leggurinn sem við skoðum
með tímann t þá setjum við d[v] = min(d[v], t) ef t ≥ d[u].


# 2.3 Verkþættir
## 2.3.1 Þáttun (⋆)
Gögnin fyrir strætó eru gefin á GTFS formi, lýsingu á GTFS er að finna á https://gtfs.
org/schedule/reference/. Til að búa til netið þarf að þátta (e. parse) gögnin, þ.e. lesa
þau inn og tengja saman á skynsamlegan hátt.

Til einföldunar ætlum við eingöngu að nota eftirfarandi hluta af gögnunum

agency_id=1 eingöngu leiðir sem tilheyra Strætó BS merktar ST.*

Leiðirnar eru ólíkar milli vikudaga, gefum okkur að við séum að vinna með virka daga
og sleppum næturstrætó (leiðir 101-106). Það má líka sleppa helgidögum, þ.e. sem er
skilgreint í calendar_dates.csv skráni.

Sumar leiðir eru í pöntunarþjónustu, við höldum eingöngu þeim leggjum sem eru með
pickup_type=0 í stop_times.csv.

#### import

In [13]:
import pandas as pd
from datetime import datetime, timedelta
import heapq

#### Hjálparföll

In [47]:
# fall til að finna út hvort við séum með tvö id's með sama nafn
def find_unique_stop_ids(df, stop_name):
    filtered_df = df[df['stop_name'] == stop_name]
    unique_stop_ids = filtered_df['stop_id'].unique()
    return unique_stop_ids

def convert_time_str_to_timedelta(time_str):
    parts = time_str.split(':')
    hours, minutes = int(parts[0]), int(parts[1])
    seconds = int(parts[2]) if len(parts) == 3 else 0
    days, hours = divmod(hours, 24)
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)

def convert_time_str_to_timedelta_2(time_str):
    # Assuming time_str is in the format "HH:MM"
    hours, minutes = map(int, time_str.split(':'))
    return timedelta(hours=hours, minutes=minutes)

def format_hours_to_time(hours):
    if hours is None:
        return None
    total_minutes = int(hours * 60)
    h, m = divmod(total_minutes, 60)
    return f"{h:02d}:{m:02d}"

def format_hours_to_time_2(timedelta_obj):
    total_seconds = int(timedelta_obj.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, _ = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}"

In [30]:
# Lesa inn gögnin
stops_df = pd.read_csv('content/stops.txt')
trips_df = pd.read_csv('content/trips.txt')
stop_times_df = pd.read_csv('content/stop_times.txt')
routes_df = pd.read_csv('content//routes.txt', comment='#')
calendar_dates_df = pd.read_csv('content/calendar_dates.txt')
# næturstrætó
night_buses = ['101', '102', '103', '104', '105', '106']

# Filter -> bara sem byrjar á ST. & agency_id = 1 & er ekki næturstrætó
routes_df = routes_df[(routes_df['route_id'].str.startswith("ST.", na=False)) &
                      (routes_df['agency_id'] == 1) &
                      (~routes_df['route_short_name'].isin(night_buses))]

# Bæta við dálki sem hefur aðeins upplýsingar um vikudaga.
trips_df['day_pattern'] = trips_df['service_id'].apply(lambda x: x.split('_')[1])

# Filter -> bara virkir dagar.
def all_weekdays_no_weekends(pattern):
    weekdays = 'MTWTF'
    return all(day in pattern for day in weekdays if day != '-')
    
filtered_trips_df = trips_df[trips_df['day_pattern'].apply(all_weekdays_no_weekends)]

# Filter -> pickup_type = 0
filtered_stop_times_df = stop_times_df[stop_times_df['pickup_type'] == 0]

# Merge -> filtered_stop_times & stops til að fá nafn á stoppi
stop_times_with_stops_df = pd.merge(filtered_stop_times_df, stops_df, on='stop_id')

# Merge -> stop_times_with_stops & trips til að fá upplýsingar um ferðir
stop_times_with_stops_and_trips_df = pd.merge(stop_times_with_stops_df, filtered_trips_df, on='trip_id')

# Merge -> Henda þessu í öllu í eina klessu
stop_times_with_stops_trips_and_routes_df = pd.merge(stop_times_with_stops_and_trips_df, routes_df[['route_id', 'route_short_name']], on='route_id')

# Merge -> stop_times_with_stops_trips_and_routes & calendar_dates til að fá upplýsingar um dagsetningar
filtered_final_df = pd.merge(stop_times_with_stops_trips_and_routes_df, calendar_dates_df, on='service_id')

# Filter -> bara fyrir daginn 20240411
filtered_final_current_df = filtered_final_df[filtered_final_df['date'] == 20240411]

# Filter -> óþarfa gögn
df_re_filtered = filtered_final_current_df.drop(['stop_headsign', 'trip_short_name', 'exception_type', 'service_id', 'route_id', 'shape_id', 'date', 'day_pattern', 'pickup_type','location_type'], axis=1)

df = df_re_filtered.sort_values(by=['direction_id', 'departure_time', 'route_short_name'])

# Add 'next_stop_id' by shifting 'stop_id' up by one within each 'trip_id' group
df['next_stop_id'] = df.groupby('trip_id')['stop_id'].shift(-1)

# Add 'next_departure_time' by shifting 'departure_time' up by one within each 'trip_id' group
df['next_departure_time'] = df.groupby('trip_id')['departure_time'].shift(-1)

display(df)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,direction_id,block_id,route_short_name,next_stop_id,next_departure_time
3729546,525756,06:29:00,06:29:00,90000397,1,Árvað,64.101159,-21.778086,Norðlingaholt,0,4176_5-A,5,90000375.0,06:29:00
3729639,525756,06:29:00,06:29:00,90000375,2,Bjallavað,64.102854,-21.776629,Norðlingaholt,0,4176_5-A,5,90000391.0,06:30:00
3729732,525756,06:30:00,06:30:00,90000391,3,Ferjuvað,64.103829,-21.771664,Norðlingaholt,0,4176_5-A,5,90000321.0,06:30:00
3729825,525756,06:30:00,06:30:00,90000321,4,Helluvað,64.102624,-21.766771,Norðlingaholt,0,4176_5-A,5,90000322.0,06:31:00
3729918,525756,06:31:00,06:31:00,90000322,5,Kambavað,64.100787,-21.765827,Norðlingaholt,0,4176_5-A,5,90000335.0,06:31:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343881,524939,24:18:00,24:18:00,90000001,37,Lækjartorg B,64.147505,-21.934808,Hlemmur,1,4121_1-A,1,90000002.0,24:19:00
343974,524939,24:19:00,24:19:00,90000002,38,Þjóðleikhúsið,64.146762,-21.931048,Hlemmur,1,4121_1-A,1,90000822.0,24:20:00
344067,524939,24:20:00,24:20:00,90000822,39,Bíó Paradís,64.145737,-21.925822,Hlemmur,1,4121_1-A,1,90000823.0,24:21:00
344160,524939,24:21:00,24:21:00,90000823,40,Barónsstígur,64.144531,-21.919734,Hlemmur,1,4121_1-A,1,90020295.0,24:22:00


Búið til net sem samsvarar leiðarkerfinu á virkum degi. Sýnið fjölda hnúta og leggja í
netinu og finnið þá stoppistöð sem hefur hæstu útgráðuna (hvaða stöð og hver er útgráðan).
Netið sem þið búið til má vera hvernig sem er svo lengi sem það sé "pythonskt"í laginu.
Þ.e. við getum ítrað yfir hnúta í netinu og fyrir hvern hnút getum við ítrað yfir granna þess
hnúts. Útfærslan í Java í Algorithms bókinni uppfyllir þessi skilyrði og eftirfarandi einfaldi
python kóði gerir það líka.

```
G = {1: [2,3], 2: [], 3: [1,2] }
for v in G:
  print(u,':')
    for u in G[u,v]:
      print(u,' -> ',v)
```
Ef við viljum bæta við einhverjum upplýsingum þá er hægt að gera það í uppflettitöflu.
Ekki er leyfilegt að nota pakka fyrir net í python á borð við networkx eða sambærilegt.

In [31]:
# búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð
all_stop_ids = df['stop_id'].unique()
stop_id_to_name = pd.Series(df.stop_name.values, index=df.stop_id).to_dict()

print(stop_id_to_name[90000397])

Árvað


#### Búum til netið G

In [43]:
G = {}
for _, row in df.iterrows():
    stop_id = row['stop_id']
    next_stop_id = row['next_stop_id']
    if pd.notna(next_stop_id):
        departure_time = convert_time_str_to_timedelta(row['departure_time'])
        next_departure_time = convert_time_str_to_timedelta(row['next_departure_time'])
        travel_time = next_departure_time - departure_time
        route_short_name = row['route_short_name']

        if stop_id not in G:
            G[stop_id] = []
        G[stop_id].append({
            'to': next_stop_id,
            'travel_time': travel_time,
            'route_short_name': route_short_name,
            'arrival_time': departure_time,  # The current stop's departure time is the next stop's arrival time
            'departure_time': next_departure_time  # Next stop's departure time
        })
        
for node, edges in list(G.items())[:5]:  # Adjust the number as needed to inspect more nodes
    print(f"Node: {node}, Stop Name: {stop_id_to_name[node]}")  # Print the current node's stop name
    for edge in edges:
        print(f"  Edge to {edge['to']} (Stop Name: {stop_id_to_name[edge['to']]})")  # Print the destination node's stop name
        print(f"    Travel Time: {edge['travel_time']}")
        print(f"    Route: {edge['route_short_name']}")
        print(f"    Arrival Time: {edge['arrival_time']}")
        print(f"    Departure Time: {edge['departure_time']}")
    print()

Node: 90000397, Stop Name: Árvað
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 6:29:00
    Departure Time: 6:29:00
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 6:44:00
    Departure Time: 6:44:00
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 6:59:00
    Departure Time: 6:59:00
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 7:14:00
    Departure Time: 7:14:00
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 7:29:00
    Departure Time: 7:29:00
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 7:44:00
    Departure Time: 7:44:00
  Edge to 90000375.0 (Stop Name: Bjallavað)
    Travel Time: 0:00:00
    Route: 5
    Arrival Time: 7:59:00
    Departure Time: 7:59:00
  Edge to 90000

In [34]:
#reikna útgráðu        
out_degrees = {stop_id: len(neighbors) for stop_id, neighbors in G.items()}

# Finna stoppistöð með hæstu útgráðu
max_out_degree_stop = max(out_degrees, key=out_degrees.get)
max_out_degree = out_degrees[max_out_degree_stop]
max_out_degree_stop_name = filtered_final_current_df[filtered_final_current_df['stop_id'] == max_out_degree_stop].iloc[0]['stop_name']

print(f"Stoppistöð með hæstu útgráðu er {max_out_degree_stop_name} með útgráðu {max_out_degree}.")

Stoppistöð með hæstu útgráðu er Mjódd með útgráðu 538.


# 2.3.2 Leit (⋆⋆)
Útfærið leitarreikniritið á netinu eins og lýst var að ofan. Sýnið niðurstöðu leitarinnar þegar
við viljum komast frá Meistaravöllum (ath. það eru tvær stöðvar með þessu nafni) til FB,
kl 11:30.

In [67]:
def find_route(G, starts, destinations, start_time):
    earliest_arrival_to_destination = None
    route_info = []
    start_time_delta = convert_time_str_to_timedelta_2(start_time)
    destinations_set = set(destinations)
    entry_count = 0

    for start in starts:
        # Initialize priority queue with the start node
        pq = [(start_time_delta, entry_count, {'stop': start, 'route': None, 'departure_time': start_time_delta, 'arrival_time': None}, [])]
        entry_count += 1

        visited = {start: start_time_delta}

        while pq:
            current_time, _, current_stop_info, path = heapq.heappop(pq)
            current_stop = current_stop_info['stop']

            if current_stop in destinations_set:
                if earliest_arrival_to_destination is None or current_time < earliest_arrival_to_destination:
                    earliest_arrival_to_destination = current_time
                    route_info = path + [current_stop_info]
                break

            for edge in G.get(current_stop, []):
                next_stop = edge['to']
                travel_time_delta = edge['travel_time']  # This should already be a timedelta object
                arrival_time_at_next_stop = current_time + travel_time_delta

                if next_stop not in visited or arrival_time_at_next_stop < visited[next_stop]:
                    visited[next_stop] = arrival_time_at_next_stop
                    next_stop_info = {
                        'stop': next_stop,
                        'route': edge.get('route_short_name'),
                        'arrival_time': arrival_time_at_next_stop,  # This is already a timedelta object
                        'departure_time': current_time  # This is already a timedelta object
                    }
                    entry_count += 1  # Ensure uniqueness for heap elements
                    heapq.heappush(pq, (arrival_time_at_next_stop, entry_count, next_stop_info, path + [current_stop_info]))

    if route_info:
        # print("Route details:")
        for stop_info in route_info:
            stop_name = stop_id_to_name.get(stop_info['stop'], 'Unknown Stop')
            route_info_str = f"Arrive at {stop_name}"
            if stop_info['route']:
                route_info_str += f" on route {stop_info['route']}"
            if stop_info['arrival_time']:
                formatted_arrival = format_hours_to_time_2(stop_info['arrival_time'])
                route_info_str += f", Arrival: {formatted_arrival}"
            if stop_info['departure_time']:
                formatted_departure = format_hours_to_time_2(stop_info['departure_time'])
                route_info_str += f", Departure: {formatted_departure}"
            # print(earliest_arrival_to_destination)
            print(route_info_str)

    return earliest_arrival_to_destination



meistaravellir = find_unique_stop_ids(df, 'Meistaravellir')
fb = find_unique_stop_ids(df, 'FB')

find_route(G, meistaravellir, fb,'11:30')

Arrive at Meistaravellir, Departure: 11:30
Arrive at Grandavegur on route 15, Arrival: 11:30, Departure: 11:30
Arrive at Reynimelur on route 15, Arrival: 11:31, Departure: 11:30
Arrive at Hagi on route 15, Arrival: 11:32, Departure: 11:31
Arrive at Kvisthagi on route 15, Arrival: 11:33, Departure: 11:32
Arrive at Dunhagi on route 15, Arrival: 11:34, Departure: 11:33
Arrive at Stúdentagarðar on route 15, Arrival: 11:35, Departure: 11:34
Arrive at Skerjagarðar on route 15, Arrival: 11:36, Departure: 11:35
Arrive at Þorragata on route 15, Arrival: 11:36, Departure: 11:36
Arrive at Reykjavíkurflugvöllur on route 15, Arrival: 11:37, Departure: 11:36
Arrive at Gamla Hringbraut on route 15, Arrival: 11:40, Departure: 11:37
Arrive at Læknagarður on route 3, Arrival: 11:41, Departure: 11:40
Arrive at Klambratún on route 3, Arrival: 11:43, Departure: 11:41
Arrive at Hlíðar on route 1, Arrival: 11:44, Departure: 11:43
Arrive at Kringlan on route 6, Arrival: 11:46, Departure: 11:44
Arrive at Skeif

datetime.timedelta(seconds=43320)


## 2.3.3 Framsetning (⋆)
Takið löglega leið og búið til leiðarlýsingu, þ.e. hvaða vagn á að taka klukkan hvað og hvar
á að skipta um stöð, svipað og gert er á myndinni að ofan.

In [ ]:

# class Node:  # Hnutur
#     def __init__(self, name: str):
#         self.name = name  # Name of the stop

# class Edge:  # Leggur
#     def __init__(self, start: Node, end: Node, start_time: str, end_time: str):
#         self.start = start  # Starting stop
#         self.end = end  # Ending stop
#         self.start_time = start_time  # Departure time from the start stop
#         self.end_time = end_time  # Arrival time at the end stop


## 2.3.4 Labb (⋆⋆)
Sumar stoppistöðvar eru nálægt hvor annari en ekki hægt að taka strætó á milli þeirra. Breytið netinu og reikniritinu ykkar til að höndla þetta tilfelli, notið GPS hnitin sem eru gefin í gögnunum og veljið hæfilega fjarlægð sem þið eruð til í að láta fólk labba. T.d. er hægt að labba úr 13 í Suðurveri yfir í 4 á Kringlumýrarbraut en 13 stoppar ekki þar. Sýnið dæmi um leið þar sem komutími styttist við að leyfa labb. Þessi aðferð við að tengja saman stöðvar er ekki fullkomin og sýnir að oft er mikilvægara að hafa aðgang að betri gögnum, t.d. eru stoppistöðvar nálægt í planinu en taka langan tíma að labba á milli.

## 2.3.5 Tímamælingar (⋆)
Mælið fjölda fyrirspurna sem aðferðin ykkar getur svarað á sekúndu. Tryggið að mælingarnar blandi stuttum og löngum leiðum með ólíkum tímum.

## 2.3.6 Bestun og gagnagrindur (⋆⋆)
Notið niðurstöður úr tímamælingu til að ná betri afköstum. Hér skiptir máli hvernig framsetningin á netinu er framkvæmd, hvaða gagnagrindur eru notaðar og hvort hægt sé að bæta eitthvað.

## 2.3.7 Færri skiptingar (⋆⋆)
Í einhverjum tilfellum eru margar ólíkar leiðir sem taka jafnlangan tíma, þ.e. seinasti vagninn kemur á áfangastað á sama tíma en fjöldi vagna sem er tekinn er ekki sá sami. Útskýrið hvernig er hægt að breyta reikniritinu til skipta sem sjaldnast um vagn og útfærið þetta á netinu. Sýnið dæmi um strætóferð þar sem niðurstaðan breytist.

## 2.3.8 A(⋆⋆⋆)
Útfærið A∗ reikniritið (sjá kafla 10.8 í https://people.mpi-inf.mpg.de/~mehlhorn/ftp/
NewToolbox/spath.pdf) með því að velja heppilegt fjarlægðarfall f . Sýnið að fjarlægðar-
fallið sem er valið sé löglegt, þ.e. reikniritið skilar alltaf réttum niðurstöðum, og framkvæmið
tímamælingar sem mæla hversu miklu þessi breyting skilar.

## 2.3.9 Pokémon (⋆ ⋆ ⋆)
Finnið þá leið sem tekur mestan fjölda vagna. Athugið að þetta verkefni er mögulega NP-erfitt, takið eftir að netið eins og það er sett upp er ekki DAG (stefnt óhringað net) en það má breyta því í DAG með því að taka út hnútana sem samsvara stoppistöðvum og bæta við hnút fyrir stoppistöð og hvern tímapunkt (x, t), þá setjum við legg frá (x, t) til (x, t′) ef t′ er næsti tímapunktur fyrir þessa stöppistöð og legg sem fara frá (x, t) til v ef vagn fer frá x á tíma t.

Finnið, með einhverju móti, lengstu mögulegu leið sem tekur sem flesta ólíka strætis-
vagnaleiðir.